In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
from keras.layers import InputLayer, Embedding, Bidirectional, LSTM, SpatialDropout1D
from keras.models import Sequential
import sys
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils import pad_sequences
import tensorflow as tf
from json import dumps
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
sys.path.append("/content/drive/MyDrive")
import load_data
import preprocessing

In [ ]:
data_loader = load_data.DataLoader()
unique_words, unique_tags, sentences = data_loader.open_csv("/content/drive/MyDrive/ner_dataset.csv")
preprocessor = preprocessing.Preprocessor(35)
X, y = preprocessor.process(sentences, unique_words, unique_tags)

In [ ]:
with open('/content/drive/MyDrive/word_index.json', 'w') as file:
  file.write(dumps(preprocessor.words_encoding))

with open('/content/drive/MyDrive/tag_index.json', 'w') as file:
  file.write(dumps(preprocessor.tags_encoding))

In [ ]:
model = Sequential()
model.add(InputLayer((35)))
model.add(Embedding(len(unique_words), 35))
model.add(SpatialDropout1D(DROPOUT))
model.add(Bidirectional(LSTM(64, return_sequences=True, recurrent_dropout=RECURRENT_DROPOUT)))

model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='LR', expand_nested=True, dpi=300,
)

In [ ]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
chkpt = ModelCheckpoint("/content/drive/MyDrive/model_weights.h5", monitor='val_loss',verbose=1, save_best_only=True, save_weights_only=False, mode='min')

early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=1, verbose=0, mode='max', baseline=None, restore_best_weights=False)

callbacks = [chkpt, early_stopping]


history = model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_test,y_test),
    batch_size=32,
    epochs=3,
    callbacks=callbacks,
    verbose=1
)